В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [ ]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [ ]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 128.1/128.1MB downloaded
Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Базовые операции с векторами**

In [ ]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [ ]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [ ]:
!pip install gensim

import gensim.downloader as api

# 1. Посмотрим список доступных датасетов
print("Доступные датасеты в gensim:")
datasets = list(api.info()['models'].keys())
for i, dataset in enumerate(datasets[:10], 1):  # Покажем первые 10
    print(f"  {i}. {dataset}")
if len(datasets) > 10:
    print(f"  ... и еще {len(datasets) - 10} датасетов")

print("\n" + "="*50 + "\n")

# 2. Выберем датасет 'glove-twitter-25'
print("Загружаю датасет 'glove-twitter-25'...")
try:
    model = api.load('glove-twitter-25')
    print("Датасет успешно загружен!")
except Exception as e:
    print(f"Ошибка при загрузке: {e}")
    print("\nПробую загрузить другой датасет...")
    # Попробуем самый маленький
    model = api.load('text8')  # text8 - очень маленький корпус для тестов

print(f"Тип объекта: {type(model)}")
print(f"Размер словаря: {len(model)} слов")
if hasattr(model, 'vector_size'):
    print(f"Размерность векторов: {model.vector_size}")

print("\n" + "="*50 + "\n")

# 3. Проверим, что можем работать с данными
if hasattr(model, 'most_similar'):
    test_words = ['apple', 'cat', 'good', 'computer', 'run']

    for word in test_words:
        if word in model:
            print(f"Найдено похожих слов для '{word}':")
            try:
                similar_words = model.most_similar(word, topn=3)
                for w, sim in similar_words:
                    print(f"  - {w}: {sim:.3f}")
                print()
            except Exception as e:
                print(f"  Ошибка: {e}")
        else:
            print(f"Слово '{word}' отсутствует в словаре\n")

print("\n" + "="*50 + "\n")

Доступные датасеты в gensim:
  1. fasttext-wiki-news-subwords-300
  2. conceptnet-numberbatch-17-06-300
  3. word2vec-ruscorpora-300
  4. word2vec-google-news-300
  5. glove-wiki-gigaword-50
  6. glove-wiki-gigaword-100
  7. glove-wiki-gigaword-200
  8. glove-wiki-gigaword-300
  9. glove-twitter-25
  10. glove-twitter-50
  ... и еще 3 датасетов


Загружаю датасет 'glove-twitter-25'...
Датасет успешно загружен!
Тип объекта: <class 'gensim.models.keyedvectors.KeyedVectors'>
Размер словаря: 1193514 слов
Размерность векторов: 25


Найдено похожих слов для 'apple':
  - windows: 0.895
  - microsoft: 0.886
  - google: 0.882

Найдено похожих слов для 'cat':
  - dog: 0.959
  - monkey: 0.920
  - bear: 0.914

Найдено похожих слов для 'good':
  - too: 0.965
  - day: 0.953
  - well: 0.950

Найдено похожих слов для 'computer':
  - camera: 0.908
  - cell: 0.892
  - server: 0.874

Найдено похожих слов для 'run':
  - down: 0.970
  - up: 0.958
  - out: 0.951





2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [ ]:
import gensim.downloader as api

# Загружаем один раз
model = api.load('glove-twitter-25')

def похожие_слова(слово, n=10):
    """Простая функция для поиска похожих слов"""
    if слово in model:
        return model.most_similar(слово, topn=n)
    return []

# Используем
print("Похожие на 'cat':")
for w, s in похожие_слова("cat", 5):
    print(f"{w} - {s:.3f}")

Похожие на 'cat':
dog - 0.959
monkey - 0.920
bear - 0.914
pet - 0.911
girl - 0.888


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [ ]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [ ]:
from gensim.models import Word2Vec

# Наши предложения про готовку
рецепты = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

# Создаем и обучаем модель
print("Создаю модель Word2Vec для анализа кулинарных текстов...")
модель = Word2Vec(
    sentences=рецепты,    # наши тексты
    vector_size=50,       # размер вектора слова
    window=3,             # смотрим на 3 слова вокруг
    min_count=1,          # берем все слова
    workers=2,            # два потока
    sg=1                  # алгоритм skip-gram
)

print("Готово! Модель обучена.")
print(f"В словаре {len(модель.wv)} кулинарных слов")
print()

# Смотрим, какие слова есть в словаре
print("Слова из наших рецептов:")
слова = list(модель.wv.key_to_index.keys())
for слово in слова:
    print(f"• {слово}")

print()
print("-" * 40)

# Ищем похожие слова
print("А теперь посмотрим на семантические связи:")

# Проверяем для глаголов готовки
глаголы = ['варить', 'жарить', 'резать', 'печь']

for глагол in глаголы:
    if глагол in модель.wv:
        print(f"\nЧто похоже на '{глагол}'?")
        похожие = модель.wv.most_similar(глагол, topn=3)

        for слово, близость in похожие:
            # Преобразуем в проценты для наглядности
            проценты = близость * 100
            print(f"  {слово} ({проценты:.1f}% схожести)")
    else:
        print(f"\nСлово '{глагол}' не нашлось в текстах")

print()
print("-" * 40)

# Сравниваем пары слов
print("Сравним некоторые слова:")

пары = [
    ('суп', 'салат'),
    ('духовка', 'сковорода'),
    ('овощи', 'фрукты')
]

for слово1, слово2 in пары:
    if слово1 in модель.wv and слово2 in модель.wv:
        схожесть = модель.wv.similarity(слово1, слово2)
        # Оценка от 0 до 1, где 1 - очень похожи
        print(f"  {слово1} и {слово2}: {схожесть:.2f}")
    else:
        print(f"  {слово1} и {слово2}: не можем сравнить")

print()
print("-" * 40)

# Покажем векторы слов
print("Векторное представление слов (первые 5 значений):")
примерные_слова = ['суп', 'овощи', 'духовка']

for слово in примерные_слова:
    if слово in модель.wv:
        вектор = модель.wv[слово]
        # Берем только первые 5 чисел
        короткий_вектор = вектор[:5]
        print(f"\n'{слово}': {короткий_вектор}")
        print(f"  (и еще {len(вектор)-5} чисел...)")

Создаю модель Word2Vec для анализа кулинарных текстов...
Готово! Модель обучена.
В словаре 65 кулинарных слов

Слова из наших рецептов:
• овощи
• мясо
• соус
• вода
• тесто
• духовка
• специи
• варить
• брокколи
• питание
• здоровое
• парить
• торт
• десерт
• сахар
• сливки
• взбивать
• холодильник
• мариновать
• чашка
• кофе
• чай
• кипятить
• яблоки
• начинка
• пирог
• месить
• тост
• бекон
• яичница
• завтрак
• готовить
• фольга
• лимон
• рыба
• запекать
• травы
• вино
• горшок
• говядина
• тушить
• барбекю
• уголь
• гриль
• соль
• паста
• молоко
• яйца
• ингредиенты
• смешивать
• огурцы
• помидоры
• салат
• резать
• дрожжи
• мука
• хлеб
• печь
• масло
• сковорода
• курица
• жарить
• картофель
• морковь
• суп

----------------------------------------
А теперь посмотрим на семантические связи:

Что похоже на 'варить'?
  вино (24.0% схожести)
  ингредиенты (21.7% схожести)
  хлеб (19.4% схожести)

Что похоже на 'жарить'?
  парить (31.1% схожести)
  взбивать (30.9% схожести)
  торт (27

In [ ]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

4. Проверьте модель

In [ ]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

In [ ]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.most_similar('варить', topn=5)  # Убрали .wv
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")
except AttributeError:
    # Если model.wv существует, используем его
    try:
        similar = model.wv.most_similar('варить', topn=5)
        print("Слова, похожие на 'варить':")
        for word, score in similar:
            print(f"  {word}: {score:.4f}")
    except:
        print("Ошибка при поиске похожих слов")

print("\n" + "="*50)

# Найдем слова, похожие на "духовка"
try:
    # Пробуем оба варианта
    try:
        similar_duhovka = model.most_similar('духовка', topn=5)
    except AttributeError:
        similar_duhovka = model.wv.most_similar('духовка', topn=5)

    print("Слова, похожие на 'духовка':")
    for word, score in similar_duhovka:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

print("\n" + "="*50)

# Найдем слова, похожие на "овощи"
try:
    # Пробуем оба варианта
    try:
        similar_ovoschi = model.most_similar('овощи', topn=5)
    except AttributeError:
        similar_ovoschi = model.wv.most_similar('овощи', topn=5)

    print("Слова, похожие на 'овощи':")
    for word, score in similar_ovoschi:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'варить':
  борщ: 0.9473
  шить: 0.9186
  жарить: 0.9179
  готовить: 0.9172
  челку: 0.9095

Слово 'духовка' не найдено в словаре

Слова, похожие на 'овощи':
  фрукты: 0.9272
  столы: 0.8663
  курицу: 0.8606
  носки: 0.8594
  зеленые: 0.8577


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [ ]:
from gensim.models import FastText

# Наши рецепты
рецепты = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

# Обучаем FastText
print("Учу модель FastText...")
ft_model = FastText(
    sentences=рецепты,  # наши тексты
    vector_size=50,     # размер вектора
    window=3,           # смотрим на 3 слова вокруг
    min_count=1,        # берем все слова
    workers=2           # два потока
)

print("Готово! Модель обучена")
print(f"Слов в словаре: {len(ft_model.wv)}")
print()

# Проверим похожие слова
print("Что похоже на 'варить'?")
if 'варить' in ft_model.wv:
    похожие = ft_model.wv.most_similar('варить', topn=5)
    for слово, балл in похожие:
        print(f"  {слово} ({балл:.3f})")

print()
print("-" * 40)

# Особенность FastText: работает с неизвестными словами
print("Особенность FastText:")
print("Может находить похожие слова, даже если их не было в текстах!")

# Слова, которых нет в исходных текстах
новые_слова = ['варил', 'овощной', 'жарильня']

for слово in новые_слова:
    print(f"\nДля слова '{слово}' (его не было в текстах):")
    try:
        похожие = ft_model.wv.most_similar(слово, topn=3)
        for похожее_слово, балл in похожие:
            print(f"  {похожее_слово} ({балл:.3f})")
    except:
        print(f"  Не получилось найти похожие слова")

print()
print("-" * 40)

# Еще примеры
print("Еще примеры:")

слова_для_проверки = ['духовка', 'овощи', 'тесто']

for слово in слова_для_проверки:
    print(f"\nПохожие на '{слово}':")
    if слово in ft_model.wv:
        похожие = ft_model.wv.most_similar(слово, topn=3)
        for похожее_слово, балл in похожие:
            print(f"  {похожее_слово} ({балл:.3f})")

print()
print("-" * 40)

# Сохраним модель
ft_model.save("моя_кулинарная_модель.model")
print("Модель сохранена в файл 'моя_кулинарная_модель.model'")

Учу модель FastText...
Готово! Модель обучена
Слов в словаре: 65

Что похоже на 'варить'?
  жарить (0.535)
  парить (0.480)
  месить (0.354)
  тушить (0.340)
  специи (0.262)

----------------------------------------
Особенность FastText:
Может находить похожие слова, даже если их не было в текстах!

Для слова 'варил' (его не было в текстах):
  уголь (0.250)
  варить (0.248)
  ингредиенты (0.207)

Для слова 'овощной' (его не было в текстах):
  овощи (0.379)
  суп (0.257)
  десерт (0.226)

Для слова 'жарильня' (его не было в текстах):
  торт (0.335)
  гриль (0.233)
  ингредиенты (0.219)

----------------------------------------
Еще примеры:

Похожие на 'духовка':
  взбивать (0.457)
  лимон (0.356)
  салат (0.305)

Похожие на 'овощи':
  жарить (0.296)
  фольга (0.257)
  морковь (0.230)

Похожие на 'тесто':
  гриль (0.341)
  яблоки (0.327)
  сковорода (0.311)

----------------------------------------
Модель сохранена в файл 'моя_кулинарная_модель.model'


6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [ ]:
# Проверяем нашу модель

print("Что наша модель думает о словах?")
print()

# 1. Для слова "варить"
print("1. Что похоже на 'варить'?")
похожие_на_варить = ft_model.wv.most_similar('варить', topn=5)
for слово, как_похоже in похожие_на_варить:
    # Переводим в проценты для понятности
    проценты = как_похоже * 100
    print(f"{слово} ({проценты:.1f}% похоже)")

print()
print("-" * 30)

# 2. Для слова "духовка"
print("2. Что похоже на 'духовка'?")
похожие_на_духовка = ft_model.wv.most_similar('духовка', topn=5)
for слово, как_похоже in похожие_на_духовка:
    проценты = как_похоже * 100
    print(f"{слово} ({проценты:.1f}% похоже)")

print()
print("-" * 30)

# 3. Для слова "овощи"
print("3. Что похоже на 'овощи'?")
похожие_на_овощи = ft_model.wv.most_similar('овощи', topn=5)
for слово, как_похоже in похожие_на_овощи:
    проценты = как_похоже * 100
    print(f"{слово} ({проценты:.1f}% похоже)")

Что наша модель думает о словах?

1. Что похоже на 'варить'?
жарить (53.5% похоже)
парить (48.0% похоже)
месить (35.4% похоже)
тушить (34.0% похоже)
специи (26.2% похоже)

------------------------------
2. Что похоже на 'духовка'?
взбивать (45.7% похоже)
лимон (35.6% похоже)
салат (30.5% похоже)
курица (30.4% похоже)
тост (29.4% похоже)

------------------------------
3. Что похоже на 'овощи'?
жарить (29.6% похоже)
фольга (25.7% похоже)
морковь (23.0% похоже)
соус (21.7% похоже)
торт (20.9% похоже)


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [ ]:
# Сравнение моделей
print("Проверим, как модели справляются с опечатками\n")

# Наши слова с ошибками
слова_с_ошибками = [
    ('варить', 'варитть'),      # лишняя буква
    ('духовка', 'духовкa'),     # латинская буква
    ('овощи', 'авощи'),         # не та первая буква
    ('сковорода', 'скаворода'), # переставлены буквы
]

for правильное, с_ошибкой in слова_с_ошибками:
    print(f"\nСлово с ошибкой: '{с_ошибкой}' (должно быть '{правильное}')")
    print("-" * 40)

    # Word2Vec
    try:
        # Пробуем найти в Word2Vec
        w2v_результат = model.wv.most_similar(с_ошибкой, topn=2)
        print(f"  Word2Vec: нашёл {[w for w, _ in w2v_результат]}")
    except:
        print(f"  Word2Vec: не нашёл такое слово")

    # FastText
    try:
        # Пробуем найти в FastText
        ft_результат = ft_model.wv.most_similar(с_ошибкой, topn=2)
        print(f"  FastText: нашёл {[w for w, _ in ft_результат]}")
    except:
        print(f"  FastText: не нашёл такое слово")

print("\n" + "=" * 50)
print("ВЫВОД: FastText лучше справляется с опечатками!")
print("Потому что он смотрит на части слов, а не на слова целиком.")

Проверим, как модели справляются с опечатками


Слово с ошибкой: 'варитть' (должно быть 'варить')
----------------------------------------
  Word2Vec: не нашёл такое слово
  FastText: нашёл ['варить', 'печь']

Слово с ошибкой: 'духовкa' (должно быть 'духовка')
----------------------------------------
  Word2Vec: не нашёл такое слово
  FastText: нашёл ['духовка', 'взбивать']

Слово с ошибкой: 'авощи' (должно быть 'овощи')
----------------------------------------
  Word2Vec: не нашёл такое слово
  FastText: нашёл ['овощи', 'фольга']

Слово с ошибкой: 'скаворода' (должно быть 'сковорода')
----------------------------------------
  Word2Vec: не нашёл такое слово
  FastText: нашёл ['сковорода', 'горшок']

ВЫВОД: FastText лучше справляется с опечатками!
Потому что он смотрит на части слов, а не на слова целиком.


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [ ]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [ ]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [ ]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [ ]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Наши рецепты
рецепты = [
    "варить суп овощи",
    "жарить курица масло",
    "печь хлеб духовка",
    "резать овощи салат",
    "смешивать тесто яйца"
]

# 1. Подготавливаем документы
подготовленные_документы = []
for i, текст in enumerate(рецепты):
    слова = текст.split()
    документ = TaggedDocument(words=слова, tags=[f"doc_{i}"])
    подготовленные_документы.append(документ)

# 2. Обучаем модель
модель = Doc2Vec(
    documents=подготовленные_документы,
    vector_size=50,
    min_count=1
)

print("Модель Doc2Vec готова!")
print(f"Обучили на {len(модель.dv)} документах\n")

# 3. Сравниваем doc_2 и doc_4
print("Сравниваем doc_2 и doc_4:")
print(f"doc_2: {рецепты[2]}")
print(f"doc_4: {рецепты[4]}")

# Считаем схожесть
схожесть = модель.dv.similarity("doc_2", "doc_4")
print(f"\nСхожесть: {схожесть:.3f}")

# Объясняем результат
if схожесть > 0.5:
    print("Документы довольно похожи!")
else:
    print("Документы не очень похожи.")

Модель Doc2Vec готова!
Обучили на 5 документах

Сравниваем doc_2 и doc_4:
doc_2: печь хлеб духовка
doc_4: смешивать тесто яйца

Схожесть: -0.042
Документы не очень похожи.


9. Найдите самый похожий документ на doc_1

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Сначала создадим документы
documents = [
    "варить суп овощи морковь картофель",
    "жарить курица сковорода масло специи",
    "печь хлеб мука дрожжи духовка",
    "резать овощи салат помидоры огурцы",
    "смешивать ингредиенты тесто яйца молоко",
    "варить паста вода соль соус",
    "гриль мясо овощи уголь барбекю",
    "тушить говядина горшок вино травы",
    "запекать рыба лимон духовка фольга",
    "готовить завтрак яичница бекон тост"
]

# Создаем размеченные документы
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

# Обучаем модель
print("Обучаю модель Doc2Vec...")
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Модель обучена!\n")

# Теперь ищем самый похожий документ на doc_1
print("=" * 50)
print("Ищем самый похожий документ на doc_1")
print("=" * 50)

# Что такое doc_1?
print(f"doc_1: '{documents[1]}'")
print()

# Ищем похожие документы
print("Похожие документы на doc_1:")
similar_docs = doc_model.dv.most_similar("doc_1", topn=len(documents))

# Показываем результаты
for i in range(1, min(4, len(similar_docs))):  # Покажем первые 3 (начиная со второго)
    doc_tag, similarity = similar_docs[i]
    doc_id = int(doc_tag.split('_')[1])

    print(f"\n{i}. {doc_tag}: {similarity:.3f}")
    print(f"   '{documents[doc_id]}'")

# Самый похожий документ
most_similar_tag, most_similar_score = similar_docs[1]
most_similar_id = int(most_similar_tag.split('_')[1])

print("\n" + "="*40)
print(f"САМЫЙ ПОХОЖИЙ ДОКУМЕНТ НА doc_1:")
print(f"doc_{most_similar_id} (схожесть: {most_similar_score:.3f})")
print(f"Текст: '{documents[most_similar_id]}'")

Обучаю модель Doc2Vec...
Модель обучена!

Ищем самый похожий документ на doc_1
doc_1: 'жарить курица сковорода масло специи'

Похожие документы на doc_1:

1. doc_0: 0.274
   'варить суп овощи морковь картофель'

2. doc_5: 0.249
   'варить паста вода соль соус'

3. doc_3: 0.204
   'резать овощи салат помидоры огурцы'

САМЫЙ ПОХОЖИЙ ДОКУМЕНТ НА doc_1:
doc_0 (схожесть: 0.274)
Текст: 'варить суп овощи морковь картофель'


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [ ]:
from gensim.models import FastText

# Данные
рецепты = [
    ['варить', 'суп', 'овощи'],
    ['жарить', 'курица', 'масло'],
    ['печь', 'хлеб', 'духовка'],
    ['резать', 'овощи', 'салат']
]

print("Сравниваем три модели FastText:")
print("="*40)

# Обучаем три модели
модели = {}
размерности = [10, 50, 100]

for размер in размерности:
    print(f"\nМодель с размерностью {размер}:")
    модель = FastText(рецепты, vector_size=размер, min_count=1)
    модели[размер] = модель
    print(f"  Обучена! Слов: {len(модель.wv)}")

print("\n" + "="*40)
print("Тестируем на слове 'варить':")
print("="*40)

for размер in размерности:
    модель = модели[размер]

    print(f"\nРазмерность {размер}:")
    if 'варить' in модель.wv:
        похожие = модель.wv.most_similar('варить', topn=2)
        for слово, оценка in похожие:
            проценты = оценка * 100
            print(f"  {слово} ({проценты:.1f}%)")
    else:
        print("  Слово не найдено")

print("\n" + "="*40)
print("Тестируем на слове 'духовка':")
print("="*40)

for размер in размерности:
    модель = модели[размер]

    print(f"\nРазмерность {размер}:")
    if 'духовка' in модель.wv:
        похожие = модель.wv.most_similar('духовка', topn=2)
        for слово, оценка in похожие:
            проценты = оценка * 100
            print(f"  {слово} ({проценты:.1f}%)")

Сравниваем три модели FastText:

Модель с размерностью 10:
  Обучена! Слов: 11

Модель с размерностью 50:
  Обучена! Слов: 11

Модель с размерностью 100:
  Обучена! Слов: 11

Тестируем на слове 'варить':

Размерность 10:
  печь (39.4%)
  жарить (35.9%)

Размерность 50:
  жарить (56.1%)
  резать (30.5%)

Размерность 100:
  жарить (57.7%)
  салат (10.5%)

Тестируем на слове 'духовка':

Размерность 10:
  жарить (61.0%)
  овощи (56.7%)

Размерность 50:
  салат (25.3%)
  масло (21.9%)

Размерность 100:
  жарить (14.2%)
  варить (6.2%)
